In [1]:
# Making a function StockVol to calibrate stock volitility under geometric Brownian motion model
!pip install pandas_datareader
!pip install yfinance
import numpy as np
import matplotlib as m
import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import data
import yfinance as yf
import datetime as dt
from datetime import date
yf.pdr_override()

/Users/nialllawler/anaconda3/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [12]:
#USE    : To get historical data for a particular stock
#INPUT  : code of the company "goog"
#OUTPUT : array of close price of data
def GetData(companyName, years=1):
    # Getting today's date
    today = date.today()
    # Setting start date to historical years
    startDate = today.replace(today.year-years)
    # Date : Open High Low Close "Adj Close" Volume
    data = pdr.get_data_yahoo(companyName, startDate, today)
    # Cleaning to get the close prices only
    closePrice = [data["Close"][i] for i in range(len(data))]
    # Return close price of the data
    return closePrice

In [13]:
#USE        : to find the volitility of the stock data
#INPUT      : array of 1 year historical prices
#ASSUMPTION : Stock doesn't pay dividends
#OUTPUT     : historical volitility of the stock (standard deviation)
def StockVol(histoPrice):
    #Finding the ratio of current/previous values
    Sn = [np.log(histoPrice[i+1]/histoPrice[i]) for i in range(len(histoPrice)-1)]
    return np.sqrt(np.var(Sn, ddof=1))

In [14]:
#USE        : to generate n stock path
#INPUT      : n-> number of stock path, sigma-> volitility, 
#           : T-> TerminalTime (yearly unit), nT-> numberOfTimePeriods
#           : r-> interest rate, delta-> continuous dividend yield 
#           : S0-> initial price of the stock
#ASSUMPTION : Stock is not paying any dividends
#OUTPUT     : Stock Path as a matrix
def StockPath(n, sigma = 0, S0=0, T=1, nT=10, r=.02, delta=0):
    path = []
    for period in range(nT):
        # Step size for every path simulation
        step =  T/nT
        #
        periodPrice = [S0]
        # Computation of n simulated stock prices 
        # np.random.normal(0,1,n) outputs array of n normal values
        Y = np.exp(((r-delta-(sigma**2)/2)*step)+(sigma*np.random.normal(0,1,nT)*np.sqrt(step)))
        # now we have array of simulated factor for n stock path 
        # we want to multiply this to previous to find simulated price
        counter = 0
        for y in Y:
            periodPrice.append(periodPrice[counter]*y)
            counter += 1
        path.append(periodPrice)
    return(path)      

In [23]:
#USE        : To generate the European put option price through Monte Carlo method
#INPUT      : n-> number of stock paths, sigma-> volitility, 
#           : T-> TerminalTime (yearly unit), nT-> numberOfTimePeriods
#           : r-> interest rate, delta-> continuous dividend yield 
#           : S0-> initial price of the stock
#ASSUMPTION : Stock is not paying any dividends
#OUTPUTS    : Discounted Payoff Vector, Price & Variance
def EurOptPrice(StockPath, n, T=1, r=.02, K = 0):
    # Last Column of StockPath is Terminal Value
    Dis_Payoff_Vec = []
    for j in range(0,n):
        St_j = StockPath[j][n]
        # Create Column of Disounted Payoffs
        Dis_Payoff_j = np.exp(-r*T)*max(0,K - St_j)
        Dis_Payoff_Vec = np.append(Dis_Payoff_Vec,Dis_Payoff_j)
        Price = np.mean(Dis_Payoff_Vec)
        Price_Var = np.var(Dis_Payoff_Vec)
    return (Dis_Payoff_Vec, Price, Price_Var)

In [24]:
HistoPrice = GetData("goog", years=1)

In [25]:
sigma = StockVol(HistoPrice)

In [31]:
StockPaths = StockPath(n = 20, sigma = sigma, S0 = HistoPrice[0], T = 1, nT = 52, r = 0.02)

In [32]:
EurOptPrice(StockPaths, n = 20, T = 1, r = 0.02, K = 1200)

(array([53.02793582, 89.68933031, 64.00136743, 66.3746875 , 69.5022493 ,
        66.21703911, 71.97480709, 89.66799871, 64.92191505, 47.09502649,
        78.90490768, 92.79974796, 46.33845459, 58.86929218, 74.17698382,
        87.10473411, 83.1964014 , 62.68252203, 79.73631186, 70.41086784]),
 70.83462901307536,
 179.10949391682996)